In [3]:
import pandas as pd
import numpy as np

url = '.\대출금리\개인사업자 대출\기업대출금리(마이너스대출-개인사업자).csv'
class read():
    credit_distribution = pd.read_csv('.\개인사업자신용등급분포.csv', encoding='cp949')
    cd = np.array(credit_distribution['구성 비율']).astype('float')
    

    def __init__(self,url):
        self.loan_dev = url

    def tidy(self):
        self.loan_dev = pd.read_csv(self.loan_dev,encoding='cp949')
        self.loan_dev['은행'] = self.loan_dev['은행'].str.strip()
        self.loan_dev.columns = ['은행','구분','1~3등급','4등급','5등급','6등급','7~10등급','평균금리','참고사항']
        self.loan_dev.drop([0],axis=0,inplace=True)
        self.loan_dev.drop(['참고사항','구분'],axis=1,inplace=True)
        self.loan_dev['평균금리']=self.loan_dev['평균금리'].replace('-',np.NaN)
        self.loan_dev.dropna(axis=0,inplace=True)
        self.loan_dev.reset_index(drop=True,inplace=True)
        self.loan_dev.replace('-',0,inplace=True)
        self.interest_grade = self.loan_dev.loc[:,'1~3등급':'7~10등급']
        return self.loan_dev

    def multi(self):
        self.a = []
        for i in range(len(self.interest_grade)):
            self.rate_grade = np.array(self.interest_grade.loc[i:i,'1~3등급':'7~10등급'], dtype='float')[0]
            #은행 상품 단위 신용 등급 별 이자율
            self.boolean = self.rate_grade.astype('bool')
            #상품의 대상 고객 
            self.ac_cd = sum(self.boolean * self.cd)
            # 전체 신용 등급 대비 타겟 고객 비율
            self.customer_ration = self.cd / self.ac_cd
            # 타겟 고객 대비 특정 신용 등급 고객 비율
            self.real_income = self.rate_grade @ self.customer_ration
            # 대출 금리에 대한 실제 고객의 가중 평균으로 구한 상품 수익률
            self.a.append(self.real_income)
        
        self.loan_dev['가중평균수익'] = self.a
        return self.loan_dev


a = read(url)
a.tidy()
a.multi()

C:\Users\LG\AppData\Local\Temp\ipykernel_11340\1727509126.py:35: RuntimeWarning: divide by zero encountered in true_divide
  self.customer_ration = self.cd / self.ac_cd
C:\Users\LG\AppData\Local\Temp\ipykernel_11340\1727509126.py:37: RuntimeWarning: invalid value encountered in matmul
  self.real_income = self.rate_grade @ self.customer_ration


,은행,1~3등급,4등급,5등급,6등급,7~10등급,평균금리,가중평균수익
0,BNK경남은행,4.24,5.65,5.16,0,11.58,4.50,5.003768
1,BNK부산은행,4.30,5.01,5.53,7.93,8.75,4.39,5.282712
2,DGB대구은행,5.10,5.86,6.93,9.68,12.90,5.85,6.488279
3,IBK기업은행,4.57,4.91,5.74,8.13,9.35,5.90,5.520333
4,KB국민은행,4.08,5.49,7.65,10.89,9.53,4.51,5.953620
5,KDB산업은행,0,0,0,0,0,4.10,NaN
6,NH농협은행,3.77,4.53,5.77,9.34,13.00,3.99,5.397409
7,SH수협은행,3.85,0,0,0,0,4.63,3.850000
8,광주은행,5.28,5.90,6.40,7.69,8.59,5.80,6.008740
9,신한은행,4.21,4.43,5.28,9.49,11.43,4.72,5.505778
